In [53]:
import pandas as pd

In [54]:
df = pd.read_csv('historical_data.csv')
print(df.head(5))

   transaction_id transaction_date transaction_time  transaction_qty store_id  \
0               1       2024-01-01         07:06:11                2      ST2   
1               2       2024-01-01         07:08:56                2      ST2   
2               3       2024-01-01         07:14:04                2      ST2   
3               4       2024-01-01         07:20:24                1      ST2   
4               5       2024-01-01         07:22:41                2      ST2   

   product_id  unit_price  
0          32         3.0  
1          57         3.1  
2          59         4.5  
3          22         2.0  
4          57         3.1  


In [55]:
# Assuming 'transaction_date' is the column with the dates
df['transaction_date'] = pd.to_datetime(
    df['transaction_date'], format='%Y-%m-%d')

# Create new feature column 'day_of_week'
df['day_of_week'] = df['transaction_date'].dt.dayofweek
df.to_csv('with_day_numbers.csv', index=False)

In [56]:
unique_records = df[['store_id', 'product_id']].drop_duplicates().sort_values(['store_id', 'product_id']).reset_index(drop=True)
print(unique_records[:])
unique_records.to_csv('sorted.csv', index=False)

    store_id  product_id
0        ST1           1
1        ST1           2
2        ST1           3
3        ST1           4
4        ST1           5
..       ...         ...
234      ST3          81
235      ST3          82
236      ST3          83
237      ST3          84
238      ST3          87

[239 rows x 2 columns]


In [57]:
# grouped_by_day = df.groupby(['store_id', 'product_id', 'transaction_date']).sum(
#     "transaction_qty")
# grouped_by_day.to_csv('grouped.csv', index=False)

In [60]:
grouped_by_day_2 = df.groupby(['store_id', 'product_id', 'transaction_date']).agg({
    "transaction_qty": "sum",
    "transaction_id": list
}).reset_index()

grouped_by_day_2.to_csv('grouped_2.csv', index=False)

In [61]:
unique_pairs = df[['store_id', 'product_id']].drop_duplicates().values

for pair in unique_pairs:
    store_id, product_id = pair
    records = grouped_by_day_2[(grouped_by_day_2['store_id'] == store_id) & (
        grouped_by_day_2['product_id'] == product_id)]
    print(records)
    records.to_csv('record_1.csv', index=False)
    # insert data to model here
    break
    # print(store_id, product_id)
    # print(records.loc[:, ['transaction_qty']])
    # Now 'records' is a DataFrame containing all records for the current 'store_id', 'product_id' pair
    # You can now convert 'records' into a suitable format and feed it into your RNN

      store_id  product_id transaction_date  transaction_qty  \
10608      ST2          32       2024-01-01                4   
10609      ST2          32       2024-01-02                8   
10610      ST2          32       2024-01-03                4   
10611      ST2          32       2024-01-05                2   
10612      ST2          32       2024-01-06                4   
...        ...         ...              ...              ...   
10752      ST2          32       2024-05-27                6   
10753      ST2          32       2024-05-28               14   
10754      ST2          32       2024-05-29                4   
10755      ST2          32       2024-05-30               14   
10756      ST2          32       2024-05-31               14   

                                          transaction_id  
10608                                            [1, 84]  
10609                               [589, 794, 977, 999]  
10610                                 [1135, 1445, 154